In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13480760359668656692, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12387410626893966770
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys

import os
import numpy as np

import keras
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Activation, Dropout,Input, concatenate, Flatten  
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.applications.vgg19 import VGG19
from keras import optimizers, metrics, models, layers
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, auc

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf
import keras.backend as K

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0
dim_ordering: tf


In [12]:
train_dir = 'data-ISBI-resized-final/train/'
validation_dir = 'data-ISBI-resized-final/validation/'
test_dir = 'data-ISBI-resized-final/test/'
test_augmented_dir = 'data-ISBI-resized-final/test-augmented/'

extracted_features_dir = "extracted_features/"

num_classes = 2
batch_size = 32
img_height, img_width = 380, 380
input_shape = (img_height, img_width, 3)
epochs = 25

In [6]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
#     validation_split= 0.25,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
#     subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
#     subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 59169 images belonging to 2 classes.
Found 18384 images belonging to 2 classes.
Found 966 images belonging to 2 classes.


In [7]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_VGG16_MobileNet_Concatenate3.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_VGG16_MobileNet_Concatenate3.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_VGG16_MobileNet_Concatenate3.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [8]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
         'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
         'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [9]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

    model = Sequential()
#     model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(num_classes, activation='softmax'))

   
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    verbose= 2,
                    callbacks=get_callbacks(params))
    
    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    print("_" * 30)
    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]

    print('Test Accuracy {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))
    print("_" * 50)
    sys.stdout.flush() 
    keras.backend.clear_session()
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [13]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [14]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=100, trials=trials)
print(best)

Parameters testing:  {'beta_2': 0.99, 'amsgrad': False, 'dropout': 0.6, 'lr': 0.001, 'beta_1': 0.4}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 8s - loss: 0.5186 - acc: 0.7634 - val_loss: 0.5196 - val_acc: 0.7672
Epoch 2/25
 - 7s - loss: 0.4969 - acc: 0.7783 - val_loss: 0.4961 - val_acc: 0.7782
Epoch 3/25
 - 7s - loss: 0.4907 - acc: 0.7791 - val_loss: 0.5621 - val_acc: 0.7396
Epoch 4/25
 - 7s - loss: 0.4876 - acc: 0.7813 - val_loss: 0.4742 - val_acc: 0.7896
Epoch 5/25
 - 7s - loss: 0.4831 - acc: 0.7840 - val_loss: 0.4949 - val_acc: 0.7792
Epoch 6/25
 - 7s - loss: 0.4794 - acc: 0.7849 - val_loss: 0.4801 - val_acc: 0.7822
Epoch 7/25
 - 7s - loss: 0.4774 - acc: 0.7863 - val_loss: 0.5128 - val_acc: 0.7553
Epoch 00007: early stopping
18384/18384 [==============================] - 1s 39us/step
Validation Accuracy: 75.5276%
Validation Loss: 0.5128294085034918
______________________________
Test Accuracy 0.7784679089026915
__________________________________________________


Epoch 4/25
 - 7s - loss: 0.4751 - acc: 0.7889 - val_loss: 0.4919 - val_acc: 0.7832
Epoch 5/25
 - 7s - loss: 0.4711 - acc: 0.7888 - val_loss: 0.4695 - val_acc: 0.7934
Epoch 6/25
 - 7s - loss: 0.4675 - acc: 0.7925 - val_loss: 0.4707 - val_acc: 0.7917
Epoch 7/25
 - 7s - loss: 0.4632 - acc: 0.7946 - val_loss: 0.4729 - val_acc: 0.7922
Epoch 8/25
 - 7s - loss: 0.4610 - acc: 0.7950 - val_loss: 0.4606 - val_acc: 0.7986
Epoch 9/25
 - 7s - loss: 0.4577 - acc: 0.7970 - val_loss: 0.4921 - val_acc: 0.7855
Epoch 10/25
 - 7s - loss: 0.4566 - acc: 0.7975 - val_loss: 0.4675 - val_acc: 0.7963
Epoch 11/25
 - 7s - loss: 0.4542 - acc: 0.7993 - val_loss: 0.4551 - val_acc: 0.8011
Epoch 12/25
 - 7s - loss: 0.4524 - acc: 0.7992 - val_loss: 0.4730 - val_acc: 0.7916
Epoch 13/25
 - 7s - loss: 0.4498 - acc: 0.8014 - val_loss: 0.4584 - val_acc: 0.8019
Epoch 14/25
 - 7s - loss: 0.4490 - acc: 0.8008 - val_loss: 0.4662 - val_acc: 0.7932
Epoch 15/25
 - 7s - loss: 0.4483 - acc: 0.8019 - val_loss: 0.4554 - val_acc: 0.801

 - 8s - loss: 0.6691 - acc: 0.6070 - val_loss: 0.6170 - val_acc: 0.6806
Epoch 2/25
 - 7s - loss: 0.6447 - acc: 0.6603 - val_loss: 0.6132 - val_acc: 0.6806
Epoch 3/25
 - 7s - loss: 0.6388 - acc: 0.6672 - val_loss: 0.6106 - val_acc: 0.6806
Epoch 4/25
 - 7s - loss: 0.6330 - acc: 0.6725 - val_loss: 0.6080 - val_acc: 0.6806
Epoch 00004: early stopping
18384/18384 [==============================] - 1s 36us/step
Validation Accuracy: 68.0592%
Validation Loss: 0.6079677778082998
______________________________
Test Accuracy 0.6780538302277432
__________________________________________________
Parameters testing:  {'beta_2': 0.9, 'amsgrad': False, 'dropout': 0.7, 'lr': 0.0001, 'beta_1': 0.8}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 8s - loss: 0.5485 - acc: 0.7365 - val_loss: 0.5027 - val_acc: 0.7759
Epoch 2/25
 - 7s - loss: 0.5029 - acc: 0.7753 - val_loss: 0.4897 - val_acc: 0.7820
Epoch 3/25
 - 7s - loss: 0.4947 - acc: 0.7807 - val_loss: 0.4863 - val_acc: 0.7833
Epoch 4/25


Epoch 7/25
 - 7s - loss: 0.4827 - acc: 0.7852 - val_loss: 0.4821 - val_acc: 0.7851
Epoch 8/25
 - 7s - loss: 0.4808 - acc: 0.7865 - val_loss: 0.4831 - val_acc: 0.7833
Epoch 9/25
 - 7s - loss: 0.4797 - acc: 0.7860 - val_loss: 0.4798 - val_acc: 0.7864
Epoch 10/25
 - 7s - loss: 0.4784 - acc: 0.7875 - val_loss: 0.4790 - val_acc: 0.7878
Epoch 11/25
 - 7s - loss: 0.4771 - acc: 0.7872 - val_loss: 0.4777 - val_acc: 0.7876
Epoch 12/25
 - 7s - loss: 0.4754 - acc: 0.7893 - val_loss: 0.4764 - val_acc: 0.7880
Epoch 13/25
 - 7s - loss: 0.4741 - acc: 0.7891 - val_loss: 0.4760 - val_acc: 0.7887
Epoch 14/25
 - 7s - loss: 0.4733 - acc: 0.7895 - val_loss: 0.4765 - val_acc: 0.7886
Epoch 15/25
 - 7s - loss: 0.4723 - acc: 0.7889 - val_loss: 0.4742 - val_acc: 0.7899
Epoch 16/25
 - 7s - loss: 0.4705 - acc: 0.7907 - val_loss: 0.4730 - val_acc: 0.7900
Epoch 17/25
 - 7s - loss: 0.4699 - acc: 0.7910 - val_loss: 0.4721 - val_acc: 0.7918
Epoch 18/25
 - 7s - loss: 0.4684 - acc: 0.7903 - val_loss: 0.4723 - val_acc: 0.

Epoch 15/25
 - 7s - loss: 0.4874 - acc: 0.7828 - val_loss: 0.4877 - val_acc: 0.7820
Epoch 16/25
 - 7s - loss: 0.4875 - acc: 0.7817 - val_loss: 0.4871 - val_acc: 0.7821
Epoch 17/25
 - 7s - loss: 0.4869 - acc: 0.7827 - val_loss: 0.4862 - val_acc: 0.7831
Epoch 18/25
 - 7s - loss: 0.4862 - acc: 0.7824 - val_loss: 0.4857 - val_acc: 0.7833
Epoch 19/25
 - 7s - loss: 0.4847 - acc: 0.7844 - val_loss: 0.4851 - val_acc: 0.7833
Epoch 20/25
 - 7s - loss: 0.4842 - acc: 0.7838 - val_loss: 0.4848 - val_acc: 0.7838
Epoch 21/25
 - 7s - loss: 0.4839 - acc: 0.7846 - val_loss: 0.4842 - val_acc: 0.7836
Epoch 22/25
 - 7s - loss: 0.4835 - acc: 0.7847 - val_loss: 0.4837 - val_acc: 0.7839
Epoch 23/25
 - 7s - loss: 0.4821 - acc: 0.7849 - val_loss: 0.4840 - val_acc: 0.7839
Epoch 24/25
 - 7s - loss: 0.4815 - acc: 0.7859 - val_loss: 0.4827 - val_acc: 0.7845
Epoch 25/25
 - 7s - loss: 0.4817 - acc: 0.7858 - val_loss: 0.4824 - val_acc: 0.7850
18384/18384 [==============================] - 1s 37us/step
Validation Accur

...

Epoch 5/25
 - 7s - loss: 0.4730 - acc: 0.7881 - val_loss: 0.5018 - val_acc: 0.7786
Epoch 6/25
 - 7s - loss: 0.4702 - acc: 0.7899 - val_loss: 0.4642 - val_acc: 0.7966
Epoch 7/25
 - 7s - loss: 0.4668 - acc: 0.7922 - val_loss: 0.4865 - val_acc: 0.7879
Epoch 8/25
 - 7s - loss: 0.4628 - acc: 0.7939 - val_loss: 0.5055 - val_acc: 0.7690
Epoch 9/25
 - 7s - loss: 0.4605 - acc: 0.7965 - val_loss: 0.4591 - val_acc: 0.7967
Epoch 10/25
 - 7s - loss: 0.4595 - acc: 0.7957 - val_loss: 0.4941 - val_acc: 0.7773
Epoch 11/25
 - 7s - loss: 0.4570 - acc: 0.7976 - val_loss: 0.4628 - val_acc: 0.7950
Epoch 12/25
 - 7s - loss: 0.4545 - acc: 0.7989 - val_loss: 0.4891 - val_acc: 0.7819
Epoch 00012: early stopping
18384/18384 [==============================] - 1s 37us/step
Validation Accuracy: 78.1930%
Validation Loss: 0.48908931310017073
______________________________
Test Accuracy 0.8923395445134575
__________________________________________________
Parameters testing:  {'beta_2': 0.8, 'amsgrad': True, 'dropout'

Epoch 15/25
 - 7s - loss: 0.4578 - acc: 0.7986 - val_loss: 0.4503 - val_acc: 0.8052
Epoch 16/25
 - 7s - loss: 0.4569 - acc: 0.8001 - val_loss: 0.4567 - val_acc: 0.7995
Epoch 17/25
 - 7s - loss: 0.4556 - acc: 0.7989 - val_loss: 0.4568 - val_acc: 0.7998
Epoch 18/25
 - 7s - loss: 0.4557 - acc: 0.7994 - val_loss: 0.4653 - val_acc: 0.7945
Epoch 00018: early stopping
18384/18384 [==============================] - 1s 36us/step
Validation Accuracy: 79.4495%
Validation Loss: 0.46529481546374585
______________________________
Test Accuracy 0.901656314699793
__________________________________________________
Parameters testing:  {'beta_2': 0.7, 'amsgrad': True, 'dropout': 0.5, 'lr': 1e-07, 'beta_1': 0.6}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 7s - loss: 0.6485 - acc: 0.6447 - val_loss: 0.6322 - val_acc: 0.6806
Epoch 2/25
 - 7s - loss: 0.6434 - acc: 0.6560 - val_loss: 0.6293 - val_acc: 0.6806
Epoch 3/25
 - 7s - loss: 0.6408 - acc: 0.6591 - val_loss: 0.6273 - val_acc: 0.680

Epoch 7/25
 - 7s - loss: 0.4609 - acc: 0.7961 - val_loss: 0.4731 - val_acc: 0.7918
Epoch 8/25
 - 7s - loss: 0.4574 - acc: 0.7962 - val_loss: 0.4604 - val_acc: 0.7990
Epoch 9/25
 - 7s - loss: 0.4548 - acc: 0.7987 - val_loss: 0.4735 - val_acc: 0.7886
Epoch 10/25
 - 7s - loss: 0.4531 - acc: 0.7985 - val_loss: 0.4577 - val_acc: 0.8008
Epoch 11/25
 - 7s - loss: 0.4506 - acc: 0.7995 - val_loss: 0.4568 - val_acc: 0.8011
Epoch 12/25
 - 7s - loss: 0.4500 - acc: 0.8018 - val_loss: 0.4921 - val_acc: 0.7809
Epoch 13/25
 - 7s - loss: 0.4479 - acc: 0.8020 - val_loss: 0.4610 - val_acc: 0.8008
Epoch 14/25
 - 7s - loss: 0.4454 - acc: 0.8033 - val_loss: 0.4576 - val_acc: 0.8024
Epoch 15/25
 - 7s - loss: 0.4439 - acc: 0.8038 - val_loss: 0.4766 - val_acc: 0.7876
Epoch 16/25
 - 7s - loss: 0.4428 - acc: 0.8054 - val_loss: 0.4614 - val_acc: 0.7966
Epoch 17/25
 - 7s - loss: 0.4412 - acc: 0.8063 - val_loss: 0.4610 - val_acc: 0.7968
Epoch 00017: early stopping
18384/18384 [==============================] - 1s 3

Epoch 4/25
 - 7s - loss: 0.4826 - acc: 0.7851 - val_loss: 0.4826 - val_acc: 0.7849
Epoch 5/25
 - 7s - loss: 0.4790 - acc: 0.7864 - val_loss: 0.4785 - val_acc: 0.7876
Epoch 6/25
 - 7s - loss: 0.4761 - acc: 0.7885 - val_loss: 0.4757 - val_acc: 0.7884
Epoch 7/25
 - 7s - loss: 0.4734 - acc: 0.7887 - val_loss: 0.4743 - val_acc: 0.7907
Epoch 8/25
 - 7s - loss: 0.4709 - acc: 0.7896 - val_loss: 0.4745 - val_acc: 0.7907
Epoch 9/25
 - 7s - loss: 0.4687 - acc: 0.7918 - val_loss: 0.4722 - val_acc: 0.7907
Epoch 10/25
 - 8s - loss: 0.4663 - acc: 0.7931 - val_loss: 0.4735 - val_acc: 0.7907
Epoch 11/25
 - 7s - loss: 0.4645 - acc: 0.7936 - val_loss: 0.4678 - val_acc: 0.7937
Epoch 12/25
 - 7s - loss: 0.4621 - acc: 0.7943 - val_loss: 0.4665 - val_acc: 0.7940
Epoch 13/25
 - 7s - loss: 0.4608 - acc: 0.7961 - val_loss: 0.4649 - val_acc: 0.7951
Epoch 14/25
 - 7s - loss: 0.4585 - acc: 0.7975 - val_loss: 0.4658 - val_acc: 0.7956
Epoch 15/25
 - 7s - loss: 0.4569 - acc: 0.7969 - val_loss: 0.4628 - val_acc: 0.797

Epoch 10/25
 - 7s - loss: 0.4484 - acc: 0.8012 - val_loss: 0.4557 - val_acc: 0.8000
Epoch 11/25
 - 7s - loss: 0.4473 - acc: 0.8018 - val_loss: 0.4589 - val_acc: 0.7998
Epoch 00011: early stopping
18384/18384 [==============================] - 1s 38us/step
Validation Accuracy: 79.9826%
Validation Loss: 0.4588983143615764
______________________________
Test Accuracy 0.8726708074534162
__________________________________________________
Parameters testing:  {'beta_2': 0.995, 'amsgrad': True, 'dropout': 0.4, 'lr': 1e-07, 'beta_1': 0.8}
Train on 59169 samples, validate on 18384 samples
Epoch 1/25
 - 8s - loss: 0.7036 - acc: 0.5057 - val_loss: 0.6794 - val_acc: 0.6805
Epoch 2/25
 - 7s - loss: 0.6802 - acc: 0.5688 - val_loss: 0.6609 - val_acc: 0.6806
Epoch 3/25
 - 7s - loss: 0.6642 - acc: 0.6122 - val_loss: 0.6480 - val_acc: 0.6806
Epoch 4/25
 - 7s - loss: 0.6539 - acc: 0.6376 - val_loss: 0.6392 - val_acc: 0.6806
Epoch 5/25
 - 7s - loss: 0.6460 - acc: 0.6530 - val_loss: 0.6329 - val_acc: 0.680